<a href="https://colab.research.google.com/github/sp-mujuni/Zero-Burn-algorithm/blob/main/Zer0_Burn_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import datetime
from typing import List, Dict
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

class Developer:
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role
        self.tasks: List[Dict] = []
        self.hours_worked: Dict[datetime.date, float] = {}
        self.burnout_rate: Dict[datetime.date, float] = {}  # 0-1 scale, 1 being complete burnout
        self.ai_assistance_enabled = False  # Attribute for AI assistance

class BurnoutPreventionSystem:
    def __init__(self, data_file: str):
        self.developers: List[Developer] = []
        self.hours_model = LinearRegression()
        self.workload_model = LinearRegression()
        self.scaler = StandardScaler()
        self.data = pd.read_csv('developer_data.csv')

    def add_developer(self, developer: Developer):
        self.developers.append(developer)

    def prepare_data(self):
        X = self.data[['weekly_hours', 'weekly_burnout', 'weekly_tasks']]
        y_hours = self.data['next_week_hours']
        y_workload = self.data['next_week_tasks']
        return X, y_hours, y_workload

    def train_models(self):
        X, y_hours, y_workload = self.prepare_data()
        X = self.scaler.fit_transform(X)

        # Train hours model
        X_train, X_test, y_hours_train, y_hours_test = train_test_split(X, y_hours, test_size=0.2, random_state=42)
        self.hours_model.fit(X_train, y_hours_train)
        hours_pred = self.hours_model.predict(X_test)
        hours_mse = mean_squared_error(y_hours_test, hours_pred)
        print(f"Hours model MSE: {hours_mse:.2f}")

        # Train workload model
        X_train, X_test, y_workload_train, y_workload_test = train_test_split(X, y_workload, test_size=0.2, random_state=42)
        self.workload_model.fit(X_train, y_workload_train)
        workload_pred = self.workload_model.predict(X_test)
        workload_mse = mean_squared_error(y_workload_test, workload_pred)
        print(f"Workload model MSE: {workload_mse:.2f}")

    def predict_optimal_conditions(self, weekly_hours: float, weekly_burnout: float, weekly_tasks: int):
        recent_data = np.array([[weekly_hours, weekly_burnout, weekly_tasks]])
        recent_data = self.scaler.transform(recent_data)
        optimal_hours = self.hours_model.predict(recent_data)[0]
        optimal_workload = self.workload_model.predict(recent_data)[0]
        return optimal_hours, optimal_workload

    def get_recommendations(self, developer: Developer):
        recent_weekly_hours = np.mean(list(developer.hours_worked.values())[-7:])
        recent_burnout = np.mean(list(developer.burnout_rate.values())[-7:])
        recent_tasks = len([task for task in developer.tasks if task['deadline'] > datetime.date.today()])

        optimal_hours, optimal_workload = self.predict_optimal_conditions(recent_weekly_hours, recent_burnout, recent_tasks)

        recommendations = []

        if recent_weekly_hours > optimal_hours:
            recommendations.append(f"Reduce weekly working hours from {recent_weekly_hours:.1f} to approximately {optimal_hours:.1f} hours")
        elif recent_weekly_hours < optimal_hours:
            recommendations.append(f"You can potentially increase your weekly hours from {recent_weekly_hours:.1f} to approximately {optimal_hours:.1f} hours without risking burnout")

        if recent_tasks > optimal_workload:
            recommendations.append(f"Reduce your weekly tasks from {recent_tasks} to approximately {optimal_workload:.0f} tasks")
        elif recent_tasks < optimal_workload:
            recommendations.append(f"You can potentially handle more tasks. Consider increasing from {recent_tasks} to approximately {optimal_workload:.0f} tasks per week")

        if recent_burnout > 0.7:
            recommendations.append("Your recent burnout rate is high. Consider taking some time off or reducing workload significantly")
        elif recent_burnout > 0.5:
            recommendations.append("Your burnout rate is moderate. Monitor closely and consider implementing stress-reduction techniques")

        return recommendations

    def monitor_burnout(self):
        for developer in self.developers:
            recommendations = self.get_recommendations(developer)
            print(f"\nRecommendations for {developer.name}:")
            for recommendation in recommendations:
                print(f"- {recommendation}")

# Example usage
if __name__ == "__main__":
    # Assume we have a CSV file named 'developer_data.csv' with columns:
    # weekly_hours, weekly_burnout, weekly_tasks, next_week_hours, next_week_tasks
    bps = BurnoutPreventionSystem('developer_data.csv')

    # Train the models
    bps.train_models()

    # Add a developer
    dev = Developer("John Doe", "Software Engineer")
    bps.add_developer(dev)

    # Simulate some data for the developer
    for _ in range(30):
        date = datetime.date.today() - datetime.timedelta(days=_)
        dev.hours_worked[date] = np.random.randint(6, 11)
        dev.burnout_rate[date] = min(1, max(0, np.random.normal(0.5, 0.2)))

    # Add some tasks
    for _ in range(10):
        deadline = datetime.date.today() + datetime.timedelta(days=np.random.randint(1, 14))
        dev.tasks.append({
            "name": f"Task {_ + 1}",
            "deadline": deadline,
            "estimated_hours": np.random.randint(2, 8),
        })

    # Monitor burnout and provide recommendations
    bps.monitor_burnout()

Hours model MSE: 867.33
Workload model MSE: 141.69

Recommendations for John Doe:
- You can potentially increase your weekly hours from 7.7 to approximately 39.5 hours without risking burnout
- You can potentially handle more tasks. Consider increasing from 10 to approximately 36 tasks per week


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
